In [1]:
# IMPORTS
import pandas as pd
import numpy as np
import os

/tmp/ipykernel_45690/88106150.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Fragment of code to parse the CRC.SW.mRNA.symbol.count.txt file, correct its formatting, and save it as a csv that can be read straight in the analysis.

In [68]:
## LOAD TRANSCRIPTOMICS DATA AND DO BASIC FORMATTING.
## GENERATE CSV FOR FURTHER ANALYSIS.

## Move out of the notebook folder to access datasets
working_dir = os.getcwd()
working_dir = working_dir.strip('notebooks')

# Prepare all the directories and files
data_dir = working_dir + 'data/PROTECTED_DATA/BGI_Expression_Data/'
mrna_count = data_dir + 'CRC.SW.mRNA.symbol.count.txt'

mrna_count_raw = pd.read_csv(mrna_count, sep="\t", index_col=0)
mrna_count_raw = mrna_count_raw.T
mrna_count_raw.reset_index(inplace=True)
mrna_count_raw.rename(columns={'index':'Sample_ID'}, inplace=True)


## Save the dataset as a file with correct formating
correctformating = data_dir + 'CRC.SW.mRNA.symbol.count.csv'
mrna_count_raw.to_csv(correctformating, sep=',', index=False) 

Read and extract a subset of traits from the key-table, in order to select what clinical traits we want to study.

In [73]:
## LOAD KEY-DATA AND SELECT CLINICAL TRAITS

## Move out of the notebook folder to access datasets
working_dir = os.getcwd()
working_dir = working_dir.strip('notebooks')

# Prepare all the directories and files
data_dir = working_dir + 'data/PROTECTED_DATA/BGI_Expression_Data/'
key_table_dir = data_dir + 'Supplementary_Table_01.xlsx'


## Load the datasets and fix formatting
key_table = pd.read_excel(key_table_dir, header=2)
key_table.head()

## Subset the clincal traits we want to study
key_table_subset = key_table[['RNA Tumor Sample Barcode', 'Gender', 'Primary Site Disease', 
                              'Histology Subtype', 'Tumour Stage']]
key_table_subset = key_table_subset.rename(columns={'RNA Tumor Sample Barcode':'Sample_ID'})




## Save the dataset as a file with correct formating
correctformating = data_dir + 'Sample_Info_Selection.csv'
key_table_subset.to_csv(correctformating, sep=',', index=False, encoding='utf-8')

In [8]:
## Get the values of each trait for coloring in WGCNA analysis
sample_info_selection = data_dir + 'Sample_Info_Selection.csv'
sample_info_tes = pd.read_csv(sample_info_selection)

# Use set() to eliminate duplicate values in column 'C'
unique_values_set = set(sample_info_tes['Tumour Stage'])
 
# Print the unique values
print(unique_values_set)

{'Stage III', 'Stage II', 'Stage I', 'Stage IV'}


DEALING WITH INEQUAL NUMBER OF SAMPLES IN DATASET AND EXCEL

In [2]:
### LOADING REAL UNPUBLISHED DATA    -     NO PUSHING FOR THE RESULTS

## Move out of the notebook folder to access datasets
working_dir = os.getcwd()
working_dir = working_dir.strip('notebooks')

# Prepare all the files for loading
data_dir = working_dir + 'data/PROTECTED_DATA/BGI_Expression_Data/'
mrna_count = data_dir + 'CRC.SW.mRNA.symbol.count.csv'
sample_info_selection = data_dir + 'Sample_Info_Selection.csv'

In [3]:
genee = pd.read_csv(mrna_count)  
genee

,Sample_ID,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,AC118549.1
0,CRC.SW.U0001.T,12,1899,43569,1,0,544,0,2168,1529,...,664,1546,335,1096,1780,6,3092,5639,5034,1922
1,CRC.SW.U0002.T,3,2448,16953,3,0,468,1,1262,1464,...,260,1130,336,1009,1862,2,1964,7174,7733,1393
2,CRC.SW.U0004.T,14,2353,12739,1,0,254,2,1914,1694,...,591,1111,459,1703,1659,0,1963,5955,5022,1665
3,CRC.SW.U0005.T,2,789,22840,7,0,330,0,1123,695,...,271,333,343,1050,1089,6,1605,4749,2827,1790
4,CRC.SW.U0006.T,2,571,5547,3,0,300,0,1070,739,...,159,374,168,495,845,6,797,5833,1954,1016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1178,CRC.SW.UM169.T,0,2017,7267,2,0,219,0,2099,2282,...,1131,1914,372,1057,1425,4,2294,6958,7091,2029
1179,CRC.SW.UM170.T,0,1883,6542,2,0,79,0,1496,2168,...,427,702,289,1008,1340,0,1468,5062,5448,1793
1180,CRC.SW.UM171.T,0,2097,1952,11,2,192,0,1122,1840,...,402,784,309,987,1162,2,1147,3962,3890,1498
1181,CRC.SW.UM172.T,0,1124,8560,8,0,341,2,1324,1353,...,814,830,440,1546,1449,6,2939,4881,3569,1797


In [4]:
test = pd.read_csv(sample_info_selection)  
test

,Sample_ID,Gender,Primary Site Disease,Histology Subtype,Tumour Stage
0,CRC.SW.U0001.T,Female,Rectum,Adenocarcinoma,Stage II
1,CRC.SW.U0002.T,Female,Colon,Adenocarcinoma,Stage III
2,CRC.SW.U0004.T,Female,Colon,Adenocarcinoma,Stage II
3,CRC.SW.U0005.T,Female,Colon,Adenocarcinoma,Stage III
4,CRC.SW.U0006.T,Male,Colon,Adenocarcinoma,Stage III
...,...,...,...,...,...
1058,CRC.SW.UM169.T,Male,Colon,Mucinous Adenocarcinoma,Stage II
1059,CRC.SW.UM170.T,Male,Colon,Mucinous Adenocarcinoma,Stage II
1060,CRC.SW.UM171.T,Male,Rectum,Adenocarcinoma,Stage III
1061,CRC.SW.UM172.T,Female,Rectum,Adenocarcinoma,Stage II


In [80]:
# Use set() to eliminate duplicate values in column 'C'
unique_values_dataset = set(genee['Sample_ID'])
unique_values_excel = set(test['Sample_ID'])

 
# Print the unique values
print(unique_values_set)

{'CRC.SW.U0492.T', 'CRC.SW.U0142.T', 'CRC.SW.U2994.T', 'CRC.SW.U0648.T', 'CRC.SW.U1454.T', 'CRC.SW.UM105.T', 'CRC.SW.UM060.T', 'CRC.SW.U0299.T', 'CRC.SW.UM061.T', 'CRC.SW.U0344.T', 'CRC.SW.UM048.T', 'CRC.SW.U3126.T', 'CRC.SW.U0532.T', 'CRC.SW.U0938.T', 'CRC.SW.U3132.T', 'CRC.SW.U0471.T', 'CRC.SW.U0260.T', 'CRC.SW.U0400.T', 'CRC.SW.U0200.T', 'CRC.SW.U3219.T', 'CRC.SW.U3213.T', 'CRC.SW.UM063.T', 'CRC.SW.U0274.T', 'CRC.SW.UM169.T', 'CRC.SW.U0457.T', 'CRC.SW.U0267.T', 'CRC.SW.U3230.T', 'CRC.SW.UM010.T', 'CRC.SW.U3162.T', 'CRC.SW.U0553.T', 'CRC.SW.U1204.T', 'CRC.SW.UM077.T', 'CRC.SW.UM108.T', 'CRC.SW.U3198.T', 'CRC.SW.U0596.T', 'CRC.SW.U1447.T', 'CRC.SW.U0390.T', 'CRC.SW.UM015.T', 'CRC.SW.U3129.T', 'CRC.SW.UM065.T', 'CRC.SW.U0163.T', 'CRC.SW.U0095.T', 'CRC.SW.U0977.T', 'CRC.SW.U3065.T', 'CRC.SW.UM091.T', 'CRC.SW.U0591.T', 'CRC.SW.UM089.T', 'CRC.SW.U0584.T', 'CRC.SW.U1111.T', 'CRC.SW.U0175.T', 'CRC.SW.U0428.T', 'CRC.SW.U0578.T', 'CRC.SW.U3247.T', 'CRC.SW.U0303.T', 'CRC.SW.U0565.T', 'CRC.SW.U